In [2]:
from pathlib import Path
from copy import deepcopy
from cjio import cityjson
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
plt.close('all')
import numpy as np
from tqdm import tqdm
import json

#### Define the parameters in the next cell, run the rest

In [27]:
size_threshold = 50
file_path = "Zurich.city.json"
cm = cityjson.load(file_path)

In [28]:
cm

CityJSON version = 1.1
EPSG = None
bbox = [ 2677116.375 1241839.025 0.000 2689381.984 1254150.950 1044.250 ]
Extensions = ['MetadataExtended']
=== CityObjects ===
|-- Building (52834)
    |-- BuildingPart (145865)
materials = False
textures = False

In [4]:
def compute_footprint_area(co):
     """Compute the area of the footprint"""
     footprint_area = 0
     for geom in co.geometry:
        # only LoD2 (or higher) objects have semantic surfaces
        if float(geom.lod) >= 2.0:
            footprints = geom.get_surfaces(type='groundsurface')

# there can be many surfaces with label 'groundsurface'
        for i,f in footprints.items():
            for multisurface in geom.get_surface_boundaries(f):
                for surface in multisurface:

                    # cast to Shapely polygon
                    shapely_poly = Polygon(surface)
                    # print(i,shapely_poly)
                    footprint_area += shapely_poly.area
     return footprint_area

In [5]:
filtered_cm_ids = []
cm_copy = deepcopy(cm)
buildings_parts = cm_copy.get_cityobjects(type=['building', 'buildingpart'])
for buildings_part in tqdm(buildings_parts.values(), total=len(buildings_parts)):
    area = compute_footprint_area(buildings_part)
    if area > 50:
        filtered_cm_ids.append(buildings_part.__dict__['id'])

100%|██████████| 198699/198699 [00:01<00:00, 154562.26it/s]


In [6]:
len(filtered_cm_ids)

44602

In [22]:
json_cm = json.load(open(file_path))
city_objects_filtered = []
new_city_objects = {}
for id, attrs in json_cm['CityObjects'].items():
    if id not in filtered_cm_ids:
        continue
    if 'children' in attrs:
        new_children = [c for c in attrs['children'] if c not in filtered_cm_ids]
        attrs['children'] = new_children
    
    new_city_objects[id] = attrs
json_cm['CityObjects'] = new_city_objects

In [23]:
json.dump(json_cm, open('Zurich.city.areaLargerThan50.json', 'w+'))

In [24]:
lessfile_path = 'Zurich.city.areaLargerThan50.json'
lesscm = cityjson.load(lessfile_path)
lesscm

CityJSON version = 1.1
EPSG = None
bbox = [ 2677116.375 1241839.025 0.000 2689381.984 1254150.950 1044.250 ]
Extensions = ['MetadataExtended']
=== CityObjects ===
materials = False
textures = False

In [32]:
for co_id, co in lesscm.cityobjects.items():
    co.attributes['fp_area'] = compute_footprint_area(co)
    lesscm.cityobjects[co_id] = co
p_out = 'zurich_output.json'
cityjson.save(lesscm, p_out)